## 9ª Rodada de questões do CEO

### Bibliotecas

In [1]:
from sqlalchemy import create_engine
import pandas as pd

### Acessando o Banco de Dados

In [2]:
# Conexão
engine = create_engine('sqlite:///db_olist.sqlite')

# cursor
cursor = engine.connect()

**1.** Qual o número de pedido com o tipo de pagamento igual a "boleto"?


In [3]:
query = """
    SELECT 
        COUNT(o.order_id) AS num_pedido 
    FROM 
        orders o 
    WHERE 
        o.order_id IN (SELECT 
                            op.order_id
                       FROM 
                            order_payments op 
                       WHERE 
                            op.payment_type = 'boleto')
"""
# Salvando a tabela
df_question_1 = pd.read_sql(query, cursor)
df_question_1.to_csv('files/diretorio_nona_rodada_questoes/question_1.csv')

df_question_1

,num_pedido
0,19784


**2.** Crie uma tabela que mostre a média de avaliações por dia, a média de preço por dia, a soma dos preços por dia, o preço mínimo por dia, o número de pedidos por dia e o número de clientes únicos que compraram no dia.

In [4]:
query = """
    SELECT 
        DATE(t2.order_approved_at)         AS date_,
        AVG(t1.num_of_reviews)             AS average_of_reviews_per_day,
        AVG(op.payment_value)              AS average_price_payment_per_day,
        SUM(op.payment_value)              AS sum_of_price_payment_per_day,
        MIN(op.payment_value)              AS min_of_price_payment_per_day,
        SUM(t2.num_of_order_approved)      AS num_of_orders_per_day,
        SUM(t2.num_of_order_by_customer)   AS num_of_order_by_customer_per_day	 
    FROM 
         (SELECT
              o.order_id, o.customer_id, o.order_approved_at, 
              COUNT(o.order_id) AS num_of_order_approved,
              COUNT(o.customer_id) AS num_of_order_by_customer
         FROM 
            orders o 
         GROUP BY
            o.order_id) AS t2 LEFT JOIN (SELECT 
                                            or2.order_id, 
                                            COUNT(or2.order_id) AS num_of_reviews  
                                        FROM  
                                            order_reviews or2 
                                        GROUP BY 
                                            or2.order_id ) AS t1 ON (t1.order_id = t2.order_id) LEFT JOIN order_payments op ON (t2.order_id = op.order_id)
    GROUP BY 
        DATE(t2.order_approved_at)
    LIMIT 70
    """

# Salvando a tabela
df_question_2 = pd.read_sql(query, cursor)
df_question_2.to_csv('files/diretorio_nona_rodada_questoes/question_2.csv')

df_question_2

,date_,average_of_reviews_per_day,average_price_payment_per_day,sum_of_price_payment_per_day,min_of_price_payment_per_day,num_of_orders_per_day,num_of_order_by_customer_per_day
0,None,1.005714,212.170114,37129.77,0.00,175,175
1,2016-09-15,1.000000,NaN,NaN,NaN,1,1
2,2016-10-04,1.000000,110.661923,2877.21,0.74,26,26
3,2016-10-05,1.000000,214.221818,2356.44,40.15,11,11
4,2016-10-06,1.000000,175.180174,20145.72,10.50,115,115
...,...,...,...,...,...,...,...
65,2017-02-25,1.000000,110.225417,5290.82,0.86,48,48
66,2017-02-26,1.000000,153.446279,6598.19,24.86,43,43
67,2017-02-27,1.000000,138.396744,5951.06,4.63,43,43
68,2017-02-28,1.000000,252.992766,11890.66,36.10,47,47


**3.** Eu gostaria de saber, por categoria, a quantidade de produtos, o tamanho médio do produto.

In [5]:
query = """
    SELECT 
        p.product_category_name, 
        COUNT(p.product_category_name) AS num_products_by_category,
        AVG(p.product_length_cm) 	   AS average_lenght_cm,
        AVG(p.product_height_cm)       AS average_height_cm,
        AVG(p.product_width_cm)        AS average_weight_cm
    FROM 
        products p 
    GROUP BY
        p.product_category_name 
    """

# Salvando a tabela
df_question_3 = pd.read_sql(query, cursor)
df_question_3.to_csv('files/diretorio_nona_rodada_questoes/question_3.csv')

df_question_3

,product_category_name,num_products_by_category,average_lenght_cm,average_height_cm,average_weight_cm
0,None,0,28.719212,15.814450,22.566502
1,agro_industria_e_comercio,74,36.527027,28.945946,25.810811
2,alimentos,82,21.292683,14.682927,17.719512
3,alimentos_bebidas,104,23.403846,15.951923,18.394231
4,artes,55,35.927273,11.800000,23.363636
...,...,...,...,...,...
69,sinalizacao_e_seguranca,93,30.505376,23.892473,20.817204
70,tablets_impressao_imagem,9,28.111111,9.444444,21.555556
71,telefonia,1134,18.432981,6.853616,13.248677
72,telefonia_fixa,116,20.646552,10.741379,16.500000


**4.** Qual o nome da categoria de produto com o maior preço de venda?

In [6]:
query = """
    
    SELECT
        tabela.product_category_name, 
        MAX(tabela.price) AS max_price
    FROM 	
        (SELECT
            p.product_id, oi.price, p.product_category_name   
        FROM 
            products p LEFT JOIN order_items oi ON (p.product_id = oi.product_id)) AS tabela
    GROUP BY 
        tabela.product_category_name  
    """

# Salvando a tabela
df_question_4 = pd.read_sql(query, cursor)
df_question_4.to_csv('files/diretorio_nona_rodada_questoes/question_4.csv')

df_question_4

,product_category_name,max_price
0,None,3980.00
1,agro_industria_e_comercio,2990.00
2,alimentos,274.99
3,alimentos_bebidas,699.90
4,artes,6499.00
...,...,...
69,sinalizacao_e_seguranca,740.00
70,tablets_impressao_imagem,889.99
71,telefonia,2428.00
72,telefonia_fixa,1790.00
